In [ ]:
#This Notebook was initially created as a bare-bones proof of concept to confirm whether or not the project was possible.
#For those that want to immediately experiment on text-generation, this is for you as it contains strictly just the code to scrape and train our dataset.

In [ ]:
#Install dependencies
!pip install --upgrade praw
!pip install --upgrade pandas
!pip install --upgrade transformers
!pip install --upgrade datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 188 kB 26.6 MB/s 
     |████████████████████████████████| 54 kB 3.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 12.3 MB/s 
     |████████████████████████████████| 101 kB 8.7 MB/s 
     |████████████████████████████████| 596 kB 51.0 MB/s 
     |████████████████████████████████| 6.6 MB 59.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 362 kB 30.3 MB/s 
     |████████████████████████████████| 140 kB 62.3 

In [ ]:
#Check for GPU
!nvidia-smi
!/usr/local/cuda/bin/nvcc --version

Tue Jul  5 03:09:01 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import praw
import pandas as pd
from datasets import Dataset, DatasetDict
#Setup a Reddit CLIENT and insert credentials here.
reddit = praw.Reddit(
    client_id="id",
    client_secret="secret",
    user_agent="agent",
)

reddit.read_only = True

 

In [ ]:


import time
from operator import attrgetter, methodcaller
submission_tags = ["subreddit", "is_original_content","over_18","spoiler", "edited", "title","url", "selftext"]
comment_tags = ["subreddit", "edited", "parent_id", "body", "permalink"]

PRAW_SUBMISSION_ID_PREFIX = "t3"
PRAW_COMMENT_ID_PREFIX = "t1"


  
def subvar_(object_, tags):
  """
  Return a dictionary consisting of key tags and whose values are the corresponding values of the attribute tag.
  In other words, return a subset of vars() where only the keys in tags are included.
  """
  values = attrgetter(*tags)(object_)
  return dict(zip(tags, values))

def scrape_(iterator, tags):
  """
  Abstracted helper for scrapping reddit. 
  In Python Context, return a list of sub-dictionaries of each element in the iterator

  In PRAW Context, return a list of valid posts containing only the relevant tags as a key value dictionary.
  """
  return [subvar_(element, tags) for element in iterator]
  
def is_comment(fullname):
  """Given a post's fullname, Return True if a post is a comment"""
  return fullname.startswith(PRAW_COMMENT_ID_PREFIX)

def is_submission(fullname):
  """Given a post's fullname, Return True if a post is a submission"""
  return fullname.startswith(PRAW_SUBMISSION_ID_PREFIX)

def scrape_submissions(username):
  """
  Scrapes the submissions of a reddit user into a list of submissions.
  By default, only meta-data for the subreddit, OC, NSFW, spoiler, title and the submission text is kept.
  """
  user = reddit.redditor(username)
  return scrape_(user.submissions.new(limit=None), submission_tags)


def append_type_(parent_ids, is_type, tags):
  """
  An abstraction helper that given an mapping of parent_ids to child comments, a post type and tags,
  append each child comment with parent attribute information designated by tags.
  """
  parent_types = [ id for id in parent_ids.keys() if is_type(id)]
  types = scrape_(reddit.info(fullnames=parent_types), tags+["fullname"])
  for type_ in types:
    #Assumption: each element of type can access the fullname via 'type_["fullname"]'
    for comment in parent_ids[type_["fullname"]]:
      comment['parent'] = type_
    type_.pop("fullname", None) #Fullname was used to connect each comment to its parent, once that is done it is no longer needed

    


def append_parents_(comments):
    """Given a list of dictionary comments, append parent information to each comment.
    Assumes that the get function can retrieve the parent_id via comment['parent_id']
    and that the set function can append parent information to each comment via dictionary assignments"""
    parent_ids = {}
    for comment in comments:
      if comment['parent_id'] in parent_ids:
        parent_ids[comment['parent_id']].append(comment)
      else:
        parent_ids[comment['parent_id']] = [comment]
    #parent_ids = { comment['parent_id']: comment for comment in comments } #An inverse mapping where a comment's parent_id value is now the key to the comment
    append_type_(parent_ids, is_comment, ["body"])
    append_type_(parent_ids, is_submission, ["title", "url", "selftext"])
    
def scrape_comments(username):
  """
  Scrapes the comments of a reddit user and converts it into a list of comments"
  By default, only meta-data for the subreddit and the comment text is kept.
  """
  user = reddit.redditor(username)
  comments = scrape_(user.comments.new(limit=None), comment_tags)
  append_parents_(comments)
  return comments

def get(username):
  submissions = scrape_submissions(username)
  comments = scrape_comments(username)
  df = pd.DataFrame(submissions+comments)
  df[["subreddit"]] = df[["subreddit"]].astype(str)
  df[["edited"]] = df[['edited']].astype(bool)
  return Dataset.from_pandas(df)


start = time.time()
dataset = get('A_Fine_Potato')
end = time.time()
print(end - start)


It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readt

57.04941987991333


In [ ]:
#Display some of the dataset attributes here

display(dataset['parent'])



[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 {'body': 'Can I get a link to the website please?',
  'selftext': None,
  'title': None,
  'url': None},
 {'body': None,
  'selftext': '[deleted]',
  'title': 'I feel that it’s unfair to me/ I could just be dramatic and privileged.',
  'url': 'https://www.reddit.com/r/JUSTNOFAMILY/comments/vier6z/i_feel_that_its_unfair_to_me_i_could_just_be/'},
 {'body': '1. It is factually wrong that you like Kylie Boomerang\n2. This meme format is critically fucked up, it\'s supposed to be "I love my Glaives. Kylie Boomerang time! I

In [ ]:

"""
Our dataset will be formatted like so:

is_original_content: Marked metadata - Evaluated to false if it does it is NaN
spoiler:  Marked metadata - Evaluated to false if it does it is NaN
over_18: Marked metadata - Evaluated to false if it does it is NaN
edited: Marked metadata - Evaluated to false if it does it is NaN
post: Whether it is a submission, comment, or link
subreddit: Subreddit name
prompt: An initial posting that gives contenxtual information about the text. For a submission, the prompt is just the title. For a comment, the prompt is the post being responded to.
The post being repsonded to can be a comment or a submission. For a comment prompt, it is the comment message. For a submission prompt, it is a new-line delimited title and self-text, if there is a self-text.
url: Marked metadata - An url providing the link to a the post, ONLY for url posts.
response: The main text of the post. For a submission, it is the self text of the post. For a link post, show the link. For a comment, it is the comment message.

Text generation and configuration can be done by modifying the initial message to contain the specified metadata of choice. Due to how next token generation works, tokens closer to the next generated word are weighed more than tokens
further away. Therefore tags like "is_original_content" whose distance is much further than "prompt": will be have less impact on the text generation, at "response". And such the tags are ordered appropriately to their priority.

This text maniuplation requires the initial text to be staticly given before text generation.
For example, you can't make changes to the "edited" tag without setting "is_orginial_content" since the text generation occurs from left to right. 
As such, we place the url tag just before the response tag as fake URLs cannot be created manually and must be generated. 
This can be somewhat subverted by creating variations of the same dataset:
is_original_content: True
subreddid: subreddit
prompt: ...

to
subreddit: subreddit
prompt: my prompt
is_original_content: ...

By reordering our tags structure, we can make the text generation fill in any tag that we so choose.
"""

from transformers import GPT2Tokenizer, GPT2LMHeadModel
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token


def line_delimited_text(tags, get, valid):
  """
  Abstract helper that returns a line-delimited text of tag retrieved via get(tag),
  on the condition that the tag is valid via valid(tag).
  """
  return "\n".join([f"{get(tag)}" for tag in tags if valid(tag)])

def remove_parent_permalinks(example):
  """
  The post a comment is replying to has no relevant url information unless they are a submisison post(non-permalinks).
  Therefore, removing a parent's permalink is advised.
  """
  if example['parent'] and not example['parent']['selftext']:
    example['parent']['url'] = None
  return example

def create_prompt(example):
  """
  Create the prompt column.
  A comment's prompt should only contain relevant information of the parents.
  A submission's prompt should only be the title.
  """
  if example['post'] == 'comment':
    text = line_delimited_text(["body", "title", "selftext"], lambda get: example['parent'][get], lambda valid: example['parent'][valid])
    return {'prompt': text}
  return {'prompt': example['title']}

def create_response(example):
  """
  Create the response column.
  A response entry should only consist of the main text, comment or submission.
  """
  if example['post'] == 'comment':
    return {'response': example['body']}
  text = line_delimited_text(["selftext"], lambda get: example[get], lambda valid: example[valid])
  return {'response': text}

def remove_permalinks(example):
  """
  Remove the permalinks of the post. A permalink is an auto-generated url of
  linking to the reddit post. Unless the link contains relevant information that
  is semantically related to the text(a link post), permalinks are non relevant
  to text generation and should be removed.
  """
  if example['post'] != 'url':
    return {'url': None}
  return example

def assign_types(example):
  """
  Create a 'post' column signfying the type of post the entry is.
  'link' - A link post
  'comment' - A comment
  'submission' - A submission
  """
  PRAW_ATTR_TO_POST_TYPES = {'url': 'link', 'body': 'comment', 'selftext': 'submission'}
  for praw_attr, post_type, in PRAW_ATTR_TO_POST_TYPES.items():
    if example[praw_attr]:
      return {'post': post_type}
  return example

def create_text(tags):
  """
  Create a text column where each tag in tags is concanetated by tag: followed by the content of each tag.
  """
  def concatenate_text(example):
    text = line_delimited_text(tags, lambda get: f"{get}: {example[get]}", lambda all_tags_valid: True)
    return {'text': text+"<|endoftext|>"} #Append <|endoftext|> token to signify EOL for GPT-2 tokenizer
  return concatenate_text
  
def concatenate_text_func(tags):
  def concatenate_text(example):
    if 'text' in example:
      return example
    text = "\n".join([f"{tag}: {example[tag]}" for tag in tags])
    return {'text': text+"<|endoftext|>"}
  return concatenate_text

def keep_nondeleted_entry(example):
  """
  For any given entry, return whether or not any component of the text was
  removed or deleted in someway.
  Removed or deleted components pollute the quality of text generation. 
  A majority deleted or removed posts in our dataset generates text that indicate
  the post was removed. Therefore removing these entries are advisable.
  """
  invalid_keywords = ["[deleted]", "[removed]"]
  for keyword in invalid_keywords:
    if keyword in example['text']:
      return False
  return True

def tokenize(example):
  """
  Standard GPT-2 Tokenizer, assuming max token length of 1024
  """
  return tokenizer(example['text'], max_length=1024, truncation=True)


    

In [ ]:
encoded_dataset = dataset.map(assign_types)
encoded_dataset = encoded_dataset.map(remove_parent_permalinks)
encoded_dataset = encoded_dataset.map(remove_permalinks)
encoded_dataset = encoded_dataset.map(create_prompt)
encoded_dataset = encoded_dataset.map(create_response)
  
#encoded_dataset = dataset.map(concatenate_text_func(submission_tags, "selftext"))

final_columns = ["is_original_content","spoiler","over_18", "edited", "post", "subreddit", "prompt", "url", "response", ]
remaining_columns = list(set(encoded_dataset.column_names) - set(final_columns))
encoded_dataset = encoded_dataset.map(create_text(final_columns))
encoded_dataset = encoded_dataset.map(tokenize)
encoded_dataset = encoded_dataset.filter(keep_nondeleted_entry)


#verify that deleted posts are not in our dataset
invalid_keywords = ["[deleted]", "[removed]"]
for text in encoded_dataset["text"]:
  for keyword in invalid_keywords:
    if keyword in text:
      print(text)

encoded_dataset = encoded_dataset.train_test_split(test_size=0.1)


  0%|          | 0/815 [00:00<?, ?ex/s]

  0%|          | 0/815 [00:00<?, ?ex/s]

  0%|          | 0/815 [00:00<?, ?ex/s]

  0%|          | 0/815 [00:00<?, ?ex/s]

  0%|          | 0/815 [00:00<?, ?ex/s]

  0%|          | 0/815 [00:00<?, ?ex/s]

  0%|          | 0/815 [00:00<?, ?ex/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:


model = GPT2LMHeadModel.from_pretrained("gpt2")



Downloading:   0%|          | 0.00/523M [00:00<?, ?B/s]

In [ ]:


from pprint import pprint

#Verify dataset split
pprint(encoded_dataset["train"]["text"][11])



('is_original_content: None\n'
 'spoiler: None\n'
 'over_18: None\n'
 'edited: False\n'
 'post: comment\n'
 'subreddit: sadcringe\n'
 'prompt: I think what this mess is trying to say is that Angel invited Justin '
 'who then got along with Angels friend so well that they stopped hanging out '
 'with Angel. So now Angel has no friends despite being the one to introduce '
 'the others to each other.\n'
 'url: None\n'
 "response: I'd say the fact Justin came sparked an argument because Justin is "
 'just too controversial. But it is hard to define the rhyme or reason of '
 "modern art, it's all up to your interpretation. That is what makes this my "
 'favorite piece of 2021\n'
 '\n'
 'Wait there is a second panel fuck<|endoftext|>')


In [ ]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForLanguageModeling
#Initiate Trainer
data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
)

training_args = TrainingArguments(output_dir="a", 
                                  evaluation_strategy="epoch", 
                                  num_train_epochs=1,
                                  per_device_train_batch_size=1)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
    data_collator = data_collator
)

In [ ]:
#Train
trainer.train()

The following columns in the training set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: response, body, prompt, url, selftext, subreddit, parent, is_original_content, edited, spoiler, title, parent_id, over_18, permalink, text, post. If response, body, prompt, url, selftext, subreddit, parent, is_original_content, edited, spoiler, title, parent_id, over_18, permalink, text, post are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 676
  Num Epochs = 1
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gr

Epoch,Training Loss,Validation Loss
1,2.376600,2.123818


Saving model checkpoint to a/checkpoint-500
Configuration saved in a/checkpoint-500/config.json
Model weights saved in a/checkpoint-500/pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: response, body, prompt, url, selftext, subreddit, parent, is_original_content, edited, spoiler, title, parent_id, over_18, permalink, text, post. If response, body, prompt, url, selftext, subreddit, parent, is_original_content, edited, spoiler, title, parent_id, over_18, permalink, text, post are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 76
  Batch size = 8


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=676, training_loss=2.339080268814719, metrics={'train_runtime': 73.4303, 'train_samples_per_second': 9.206, 'train_steps_per_second': 9.206, 'total_flos': 39337719552000.0, 'train_loss': 2.339080268814719, 'epoch': 1.0})

In [ ]:
from transformers import pipeline
#https://www.modeldifferently.com/en/2021/12/generaci%C3%B3n-de-fake-news-con-gpt-2/
generator = pipeline('text-generation', model=model, tokenizer=tokenizer, device=0, max_length=1024)


In [ ]:

initial_text = """is_original_content: False
spoiler:  False
over_18: False
edited: False
post: comment
"""
generated_texts = generator(initial_text, num_return_sequences=5, no_repeat_ngram_size=2)

for result in generated_texts:
  print('********************************')
  print(result['generated_text'])


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


********************************
is_original_content: False
spoiler:  False
over_18: False
edited: False
post: comment
subreddit: webcomics
prompt: What would you do?
url: None
response: Why would I think that's a good idea? It would be like I'd be happy on a robot to kill a super human with one simple button but with a gun, people would assume that it's actually a bad idea. Just google that and people wouldn't believe it. Its not true that guns are bad, all they do is send them away. It's only true since guns destroy their bodies and the gun isn't just a bullet. If you tried to design a computer without a button, its just one reason that we were talking about guns. My guess would have been to use the guns to destroy the computer completely. I guess even making a completely different computer would force someone to think about something.

To be honest you can think of this as a problem for real life, when it comes to robots and we are still in the early stages of automation. You might 